Outline

1. Load data 

2. Split it into train and test set, define target

3. Feature engineering: 

   - missing data: median for numerical variables that are not really gaussian-distributed and mode for categorical
   - Numerical variable Transformation: normalisation because of high rate of outliers
   - categorical encoding: OneHotEncoder because most categorical variables have few labels
4. algorithms: 
   - model training, model selection
   - model evaluation:


In [68]:
# import librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import category_encoders as ce
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

1. Load data

In [3]:
data = pd.read_csv(r'titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# remove irrelevant columns
data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=0, inplace=True)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


2. Train and Test sets

In [5]:
# Target and predictive variables
X = data.drop(data.columns[0], axis=1)
y = data[data.columns[0]]
X.shape, y.shape

((891, 7), (891,))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
test_size = 0.2,
random_state = 9)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

3. Feature Engineering

In [7]:
# Numerical and categarical variables
cat_vars = [var for var in X.columns if X[var].dtype=='O']
cat_vars = cat_vars + ['Pclass']
num_vars =[var for var in X.columns if var not in cat_vars and var !='Survived']
print(cat_vars)
num_vars

['Sex', 'Embarked', 'Pclass']


['Age', 'SibSp', 'Parch', 'Fare']

In [109]:
# Pipeline for numerical variables : fillna with median and normalize data
num_pipe = make_pipeline(
    SimpleImputer(strategy = 'median'),
    RobustScaler()
)

In [110]:
# Pipeline for categorical variables : fillna with mode and use oneHotEncoder
cat_pipe = make_pipeline(
    SimpleImputer(strategy = 'most_frequent'),
    OneHotEncoder()
)

In [111]:
# Build the feature transformation pipeline
feature_trans_pipe = make_column_transformer(
    (num_pipe, num_vars),
    (cat_pipe, cat_vars)
)

In [113]:
# Pipeline for the ML model 
model = make_pipeline(feature_trans_pipe, LogisticRegression())
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked',
                                                   'Pclass'])])),
                ('logisticregression', LogisticRegression())])

In [114]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked',
                                                   'Pclass'])])),
                ('logisticregression', LogisticRegression())])

In [115]:
model.score(X_test, y_test)

0.776536312849162

In [116]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1], dtype=int64)

In [117]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[88, 12],
       [28, 51]], dtype=int64)

In [118]:
# Pipeline of the model
model = make_pipeline(feature_trans_pipe, XGBClassifier(random_state=2020))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Em...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=2020, ...))])

In [119]:
# List of model Hyperparameters
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'xgbclassifier', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__pipeline-1', 'columntransformer__pipeline-2', 'columntransformer__pipeline-1__memory', 'columntransformer__pipeline-1__steps', 'columntransformer__pipeline-1__verbose', 'columntransformer__pipeline-1__simpleimputer', 'columntransformer__pipeline-1__robustscaler', 'columntransformer__pipeline-1__simpleimputer__add_indicator', 'columntransformer__pipeline-1__simpleimputer__copy', 'columntransformer__pipeline-1__simpleimputer__fill_value', 'columntransformer__pipeline-1__simpleimputer__keep_empty_features', 'columntransformer__pipeline-1__simpleimputer__missing_values', 'columntransformer__pipeline-1__simpleimputer__strategy', 'columntransfo

In [120]:
# Hyperparameters tuning
params = {'xgbclassifier__n_estimators':[100, 200, 500],
          'xgbclassifier__max_depth':[3, 6, 9],
          'xgbclassifier__learning_rate':[0.001, 0.01, 0.1, 1],
          'xgbclassifier__min_child_weight': [1, 3, 5, 7],
          'xgbclassifier__gamma':[0.0, 0.1, 0.2, 0.3, 0.4],
          'xgbclassifier__colsample_bytree':[0.3, 0.4, 0.5, 0.7]
          }

grid = GridSearchCV(estimator=model, 
                    param_grid=params,
                    scoring= 'accuracy',
                    cv=5)

In [121]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('robustscaler',
                                                                                          RobustScaler())]),
                                                                         ['Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          O...
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=2020, ...))]),
             param_grid={'xgbclassifier__colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                         'xgbclassifier__gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                         'xgbclassifier__learning_rate': [0.001, 0.01, 0.1, 1],
                         'xgbclassifier__max_depth': [3, 6, 9],
                         'xgbclassifier__min_child_weight': [1, 3, 5, 7],
                         'xgbclassifier__n_estimators': [100, 200, 500]},
             scoring='accuracy')

In [122]:
grid.best_score_

0.8609081059785286

In [125]:
grid.best_params_

{'xgbclassifier__colsample_bytree': 0.7,
 'xgbclassifier__gamma': 0.2,
 'xgbclassifier__learning_rate': 0.1,
 'xgbclassifier__max_depth': 3,
 'xgbclassifier__min_child_weight': 5,
 'xgbclassifier__n_estimators': 500}

In [90]:
# Complete details for the best model
print(grid.best_estimator_)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                 

In [93]:
# Grid Search results
search_results = pd.DataFrame(grid.cv_results_)
search_results.sort_values('param_xgbclassifier__n_estimators')
search_results.to_csv('xgbclassifier_results.csv')

In [126]:


model_selected = make_pipeline(feature_trans_pipe, 
                               XGBClassifier(random_state=2020, 
                                             xgbclassifier__colsample_bytree= 0.7, 
                                             xgbclassifier__gamma= 0.2, 
                                             xgbclassifier__learning_rate= 0.1, 
                                             xgbclassifier__max_depth= 3, 
                                             xgbclassifier__min_child_weight= 5, 
                                             xgbclassifier__n_estimators= 500
                                             )
                                )

In [127]:
model_selected.fit(X_train, y_train)

[22:41:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "xgbclassifier__colsample_bytree", "xgbclassifier__gamma", "xgbclassifier__learning_rate", "xgbclassifier__max_depth", "xgbclassifier__min_child_weight", "xgbclassifier__n_estimators" } are not used.



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Em...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=2020, ...))])

In [128]:
model_selected.score(X_test, y_test)

0.7821229050279329